In [11]:
# Function to recommend books
def recommend_book(book_name, num_recommendations=5):
    matched_book_name = book_name.strip().lower()
    if matched_book_name not in pivot_df.index.str.lower():
        print(f"'{book_name}' is not found in the dataset.")
        return
    
    query_index = pivot_df.index.str.lower().get_loc(matched_book_name)
    distances, indices = knn_model.kneighbors(pivot_df.iloc[query_index, :].values.reshape(1, -1), n_neighbors=num_recommendations+1)
    
    print(f"Top {num_recommendations} recommendations similar to '{book_name}':")
    for i in range(1, len(distances.flatten())):
        print(f"{pivot_df.index[indices.flatten()[i]]}")

# Test the function
book_name = "The Testament"
recommend_book(book_name)

# Select books from the DataFrame for recommendation
selected_books_df = corrected_books_df[corrected_books_df['title'].isin(['Selected Book Title 1', 'Selected Book Title 2'])]

# Call recommend_book() function for recommendations using selected books
for index, row in selected_books_df.iterrows():
    recommend_book(row['title'])

# Save the function and the model using pickle
with open('recommend_book_function.pkl', 'wb') as f:
    pickle.dump(recommend_book, f)

with open('knn_model.pkl', 'wb') as f:
    pickle.dump(knn_model, f)

print("Function and model saved successfully.")


Top 5 recommendations similar to 'The Testament':
The Street Lawyer
Four Blind Mice
Critical Mass
Star
The Writer's Idea Book
Function and model saved successfully.


In [23]:
import os
import pandas as pd
import csv
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import pickle

# Function to fix extra delimiters within quotes in a CSV file
def fix_csv(input_file, output_file, delimiter=';'):
    with open(input_file, 'r', encoding='latin-1') as infile, open(output_file, 'w', newline='', encoding='latin-1') as outfile:
        reader = csv.reader(infile, delimiter=delimiter)
        writer = csv.writer(outfile, delimiter=delimiter)

        for line_number, row in enumerate(reader, start=1):
            if len(row) != 8:
                fixed_row = []
                in_quotes = False
                for field in row:
                    if field.count('"') % 2 == 1:
                        in_quotes = not in_quotes
                    if in_quotes and field.endswith(delimiter):
                        fixed_row.append(field[:-1])
                    else:
                        fixed_row.append(field)
                if len(fixed_row) != 8:
                    print(f"Error fixing line {line_number}: {row}")
                else:
                    writer.writerow(fixed_row)
                    print(f"Fixed line {line_number}: {row} -> {fixed_row}")
            else:
                writer.writerow(row)

    print("CSV file corrected successfully.")

# Paths for the original and corrected CSV files
input_books_file = r"C:\Users\Dell\Downloads\BX-Books.csv"
output_books_file = r"C:\Users\Dell\Downloads\BX-Books-Corrected.csv"

# Fix the CSV file
fix_csv(input_books_file, output_books_file)

# Load corrected CSV file into a DataFrame
corrected_books_df = pd.read_csv(output_books_file, sep=';', encoding='latin-1')

# Load datasets
users = pd.read_csv(r"E:\semsester 6\web programming\archive (2)\BX-Users.csv", sep=";", encoding='latin-1')
ratings = pd.read_csv(r"E:\semsester 6\web programming\archive (2)\BX-Book-Ratings.csv", sep=";", encoding='latin-1')

# Preprocess data
corrected_books_df = corrected_books_df[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-L']]
corrected_books_df.rename(columns={"Book-Title": 'title',
                                    'Book-Author': 'author',
                                    "Year-Of-Publication": 'year',
                                    "Publisher": "publisher",
                                    "Image-URL-L": "image_url"}, inplace=True)

users.rename(columns={"User-ID": 'user_id',
                      'Location': 'location',
                      "Age": 'age'}, inplace=True)

ratings.rename(columns={"User-ID": 'user_id',
                         'Book-Rating': 'rating'}, inplace=True)

# Filter users with at least 200 ratings
user_counts = ratings['user_id'].value_counts()
active_users = user_counts[user_counts >= 200].index
ratings = ratings[ratings['user_id'].isin(active_users)]

# Merge datasets
merged_df = pd.merge(ratings, corrected_books_df, on='ISBN')

# Create user-item matrix
pivot_df = merged_df.pivot_table(index='title', columns='user_id', values='rating', fill_value=0)

# Fit NearestNeighbors model
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(pivot_df.values)

# Function to recommend books
def recommend_book(book_name, num_recommendations=5):
    matched_book_name = book_name.strip().lower()
    if matched_book_name not in pivot_df.index.str.lower():
        print(f"'{book_name}' is not found in the dataset.")
        return
    
    query_index = pivot_df.index.str.lower().get_loc(matched_book_name)
    distances, indices = knn_model.kneighbors(pivot_df.iloc[query_index, :].values.reshape(1, -1), n_neighbors=num_recommendations+1)
    
    print(f"Top {num_recommendations} recommendations similar to '{book_name}':")
    for i in range(1, len(distances.flatten())):
        recommended_book_index = indices.flatten()[i]
        recommended_book = corrected_books_df.iloc[recommended_book_index]
        print(f"Title: {recommended_book['title']}")
        print(f"Author: {recommended_book['author']}")
        print(f"ISBN: {recommended_book['ISBN']}")
        print(f"Image URL: {recommended_book['image_url']}")
        print("-------------------------")

# Test the function
book_name = "The Testament"
recommend_book(book_name)

# Select books from the DataFrame for recommendation
selected_books_df = corrected_books_df[corrected_books_df['title'].isin(['Selected Book Title 1', 'Selected Book Title 2'])]

# Call recommend_book() function for recommendations using selected books
for index, row in selected_books_df.iterrows():
    recommend_book(row['title'])

# Save the function and the model using pickle
download_path = r"C:\Users\Dell\Downloads"  # Change to your desired directory
with open(os.path.join(download_path, 'recommend_book_function.pkl'), 'wb') as f:
    pickle.dump(recommend_book, f)

with open(os.path.join(download_path, 'knn_model.pkl'), 'wb') as f:
    pickle.dump(knn_model, f)

print("Function and model saved successfully.")


Error fixing line 6452: ['0735201994', 'Peterman Rides Again: Adventures Continue with the Real \\J. Peterman\\" Through Life &amp', ' the Catalog Business"', 'John Peterman', '2000', 'Prentice Hall Art', 'http://images.amazon.com/images/P/0735201994.01.THUMBZZZ.jpg', 'http://images.amazon.com/images/P/0735201994.01.MZZZZZZZ.jpg', 'http://images.amazon.com/images/P/0735201994.01.LZZZZZZZ.jpg']
Error fixing line 43667: ['0330482750', 'Three Novels: \\A Strange and Sublime Address\\"', ' \\"Afternoon Raag\\"', '\\"Freedom Song\\""', 'Amit Chaudhuri', '2001', 'Pan Macmillan', 'http://images.amazon.com/images/P/0330482750.01.THUMBZZZ.jpg', 'http://images.amazon.com/images/P/0330482750.01.MZZZZZZZ.jpg', 'http://images.amazon.com/images/P/0330482750.01.LZZZZZZZ.jpg']
Error fixing line 51751: ['0413326608', 'Upon my word!: More stories from \\My word!\\" a panel game devised by Edward J. Mason &amp', ' Tony Shryane"', 'Frank Muir', '1974', 'Eyre Methuen', 'http://images.amazon.com/images/P/04

In [28]:
book_name = "Under the Black Flag: The Romance and the Reality of Life Among the Pirates"
recommend_book(book_name)

Top 5 recommendations similar to 'Under the Black Flag: The Romance and the Reality of Life Among the Pirates':
Title: Augustus a Novel
Author: Allan Massie
ISBN: 0340412240
Image URL: http://images.amazon.com/images/P/0340412240.01.LZZZZZZZ.jpg
-------------------------
Title: Year What Life Was Like At the Turn
Author: Robert Lacey
ISBN: 0316643750
Image URL: http://images.amazon.com/images/P/0316643750.01.LZZZZZZZ.jpg
-------------------------
Title: Danziger's Adventures: From Miami to Kabul
Author: Nick Danziger
ISBN: 0586090819
Image URL: http://images.amazon.com/images/P/0586090819.01.LZZZZZZZ.jpg
-------------------------
Title: Casanova Or the Art of Happiness
Author: Lydia Flem
ISBN: 0713992484
Image URL: http://images.amazon.com/images/P/0713992484.01.LZZZZZZZ.jpg
-------------------------
Title: \Blackadder\""
Author: Richard Curtis
ISBN: 0140280359
Image URL: http://images.amazon.com/images/P/0140280359.01.LZZZZZZZ.jpg
-------------------------


In [30]:
book_name = "To Kill a Mockingbird"
recommend_book(book_name)

Top 5 recommendations similar to 'To Kill a Mockingbird':
Title: The True Face of Jack the Ripper
Author: Harris Melvin
ISBN: 1854797263
Image URL: http://images.amazon.com/images/P/1854797263.01.LZZZZZZZ.jpg
-------------------------
Title: The Scrambled States of America
Author: Laurie Keller
ISBN: 0805068317
Image URL: http://images.amazon.com/images/P/0805068317.01.LZZZZZZZ.jpg
-------------------------
Title: The Heartbreaker
Author: Carly Phillips
ISBN: 0446511528
Image URL: http://images.amazon.com/images/P/0446511528.01.LZZZZZZZ.jpg
-------------------------
Title: Side effects
Author: Woody Allen
ISBN: 0394511042
Image URL: http://images.amazon.com/images/P/0394511042.01.LZZZZZZZ.jpg
-------------------------
Title: The Hero and the Crown
Author: Robin McKinley
ISBN: 0812446690
Image URL: http://images.amazon.com/images/P/0812446690.01.LZZZZZZZ.jpg
-------------------------


In [29]:
book_name = "Danziger's Adventures: From Miami to Kabul"
recommend_book(book_name)

Top 5 recommendations similar to 'Danziger's Adventures: From Miami to Kabul':
Title: Augustus a Novel
Author: Allan Massie
ISBN: 0340412240
Image URL: http://images.amazon.com/images/P/0340412240.01.LZZZZZZZ.jpg
-------------------------
Title: Year What Life Was Like At the Turn
Author: Robert Lacey
ISBN: 0316643750
Image URL: http://images.amazon.com/images/P/0316643750.01.LZZZZZZZ.jpg
-------------------------
Title: Danziger's Adventures: From Miami to Kabul
Author: Nick Danziger
ISBN: 0586090819
Image URL: http://images.amazon.com/images/P/0586090819.01.LZZZZZZZ.jpg
-------------------------
Title: Casanova Or the Art of Happiness
Author: Lydia Flem
ISBN: 0713992484
Image URL: http://images.amazon.com/images/P/0713992484.01.LZZZZZZZ.jpg
-------------------------
Title: \Blackadder\""
Author: Richard Curtis
ISBN: 0140280359
Image URL: http://images.amazon.com/images/P/0140280359.01.LZZZZZZZ.jpg
-------------------------


In [32]:
book_name = "Purity in Death"
recommend_book(book_name)

Top 5 recommendations similar to 'Purity in Death':
Title: The Pack
Author: Elisa Lynn Carbone
ISBN: 0670036196
Image URL: http://images.amazon.com/images/P/0670036196.01.LZZZZZZZ.jpg
-------------------------
Title: aol.com: How Steve Case Beat Bill Gates, Nailed the Netheads, and Made Millions in the War for the Web
Author: Kara Swisher
ISBN: 0812928962
Image URL: http://images.amazon.com/images/P/0812928962.01.LZZZZZZZ.jpg
-------------------------
Title: Die weiï¿½?ï¿½?e Rose.
Author: Inge Scholl
ISBN: 3596118026
Image URL: http://images.amazon.com/images/P/3596118026.01.LZZZZZZZ.jpg
-------------------------
Title: Golden Swan: An East Indian Tale of Love from the Mahabharata (Timeless Tales)
Author: Marianna Mayer
ISBN: 0553070541
Image URL: http://images.amazon.com/images/P/0553070541.01.LZZZZZZZ.jpg
-------------------------
Title: The Way-Paver
Author: Anne Devlin
ISBN: 0571148166
Image URL: http://images.amazon.com/images/P/0571148166.01.LZZZZZZZ.jpg
-------------------------

In [26]:
# Display the first few rows of the DataFrame
corrected_books_df


,ISBN,title,author,year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...
271352,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...
271353,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...
271354,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...
271355,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...
